### CÁC THƯ VIỆN SỬ DỤNG

In [ ]:
import numpy as np
import pandas as pd 
import os
from keras.preprocessing.image import load_img
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import ast

### ĐỌC THƯ MỤC CHỨA DỮ LIỆU ẢNH HUẤN LUYỆN

In [ ]:
path_to_data = "D:\Contest\AIFace\data\mnt\md0\projects\sami-hackathon\private\data/"
files = os.listdir(path_to_data)
size = len(files)
print("Total samples:",size)
print(files[0])

### QUAN SÁT DỮ LIỆU

In [ ]:
path_to_label = "D:/Contest/AIFace/last_label.csv"
df = pd.read_csv(path_to_label)
genders = df['gender'].tolist()

In [ ]:
gender_counts = df['gender'].value_counts()
gx = gender_counts.plot(kind='bar', color=['blue', 'pink'])

for i, v in enumerate(gender_counts):
    gx.text(i, v + 0.1, str(v), color='black', ha='center')
plt.title('Số lần xuất hiện của mỗi giá trị trong cột "gender"')
plt.xlabel('Giới tính')
plt.ylabel('Số lần xuất hiện')
plt.show()

### RÚT TRÍCH CÁC ĐẶC TRƯNG CỦA ẢNH

In [ ]:
def extract_features(images, bboxs, indexs):
  features = []
  for row, image in tqdm(enumerate(images), total=len(images)):
    img = load_img(image, grayscale=True)
    bbox = bboxs[indexs[row]]
    bbox = ast.literal_eval(bbox)
    img = img.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
    img = img.resize((64, 64))
    img = np.array(img)
    features.append(img)

  features = np.array(features)
  # Ignore this stop if using RGB
  features = features.reshape(len(features), 64, 64, 1)
  return features

In [ ]:
X = extract_features(path_to_data + df['file_name'], df['bbox'], df.index)

In [ ]:
X = X/255.0

### KHỞI TẠO BỘ HUẤN LUYỆN VÀ KIỂM THỬ

In [ ]:
gender_map = {'Male': 0, 'Female': 1}

In [ ]:
# Apply mapping to 'gender' column
y_gender = np.array(df['gender'].map(gender_map).astype(np.float64))

In [ ]:
target_gender = np.array(y_gender)

# Split the data
x_train, x_test, y_train_gender, y_test_gender = train_test_split(
    X, target_gender, test_size=0.2, shuffle=True
)
# Check the shapes after splitting
print("Training Samples:", x_train.shape[0])
print("Testing Samples:", x_test.shape[0])

In [ ]:
print("Shape of image:",X.shape)

### XÂY DỰNG MÔ HÌNH 

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras.initializers import he_uniform
from keras.constraints import MaxNorm

In [ ]:
model_gender = Sequential()

# Convolutional layers with separable convolutions
model_gender.add(SeparableConv2D(32, (3, 3), activation='relu', kernel_initializer=he_uniform(), kernel_constraint=MaxNorm(3), input_shape=(64, 64, 1)))
model_gender.add(MaxPooling2D((2, 2)))
model_gender.add(SpatialDropout2D(0.25))

model_gender.add(SeparableConv2D(64, (3, 3), activation='relu', kernel_initializer=he_uniform(), kernel_constraint=MaxNorm(3)))
model_gender.add(MaxPooling2D((2, 2)))
model_gender.add(SpatialDropout2D(0.25))

# Flatten and fully connected layers for classification
model_gender.add(Flatten())
model_gender.add(Dense(128, activation='relu', kernel_initializer=he_uniform(), kernel_constraint=MaxNorm(3)))
model_gender.add(Dense(1, activation='sigmoid'))  # Gender prediction (binary classification)

# Compile the model with an appropriate optimizer, loss function, and metrics for your task.
model_gender.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### QUAN SÁT MÔ HÌNH

In [ ]:
model_gender.summary()

### HUẤN LUYỆN MÔ HÌNH

In [ ]:
# Train the model with your data.
h1 = model_gender.fit(
    x_train,
    y_train_gender,
    validation_data=(x_test, y_test_gender),
    epochs=25,
    batch_size=128,
    shuffle=True
)

### LƯU MÔ HÌNH

In [ ]:
model_gender.save('model_gender.h5')

### KẾT QUẢ MÔ HÌNH

In [ ]:
history = h1
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy of gender')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
history = h1
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss of gender')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()